In [414]:
import pandas as pd
import numpy as np

import requests
import bs4
from bs4 import BeautifulSoup

import re
import string

from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

import category_encoders as ce

import spacy
nlp = spacy.load('en_core_web_lg')

In [4]:
!kaggle competitions download -c ds3-which-whisky

  0%|                                                | 0.00/128k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 128k/128k [00:00<00:00, 4.82MB/s]
  0%|                                               | 0.00/1.91k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 1.91k/1.91k [00:00<00:00, 3.37MB/s]
  0%|                                                | 0.00/463k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 463k/463k [00:00<00:00, 9.89MB/s]


In [117]:
sample_submission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
df = pd.read_csv('train.csv')

In [253]:
df.head()

,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.5,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.9,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.0,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.5,1.0
4,6,Davin de Kergommeaux,"After 40 years in barrels, the trademark Canad...",199.0,96,45.0,NaN


## Category Identities

It is important to note which categories refer to which. I want to use this to extract extra meaning from the description column in order to boost model performance.]

> 1 = Scotch  
> 2 = Bourbon  
> 3 = Craft  
> 4 = Canadian  


In [252]:
# Checking for class imbalance

df['category'].value_counts()

1.0    1637
2.0     449
3.0     300
4.0     200
Name: category, dtype: int64

In [89]:
df.isnull().sum()

id                0
author            0
description       0
price            63
ratingValue       0
pert_alcohol     60
category        288
dtype: int64

Before digging into the primary features, namely the description. I'd like to look at whether there is any noteable difference between ratings, alcohol percentage and price. If the primary statistical measurements show any significant variation, it would be worth it to include them in the modeling. 

In [287]:
def price_info(data):
    print('###################')
    print('mean:   ', data.mean())
    print('median: ', data.median())
    print('mode :  ', data.mode()[0])
    print('StDev:  ', data.std())
    print()

In [288]:
print('###### Prices ########')
for i in np.arange(1,5):
    print('Categeory  = ', i)
    price_info(df[df['category'] == i]['price'])

print('\n\n###### Ratings ########')
for i in np.arange(1,5):
    print('Categeory  = ', i)
    price_info(df[df['category'] == i]['ratingValue'])
    
print('\n\n###### % Alcohol ########')
for i in np.arange(1,5):
    print('Categeory  = ', i)
    price_info(df[df['category'] == i]['pert_alcohol'])

###### Prices ########
Categeory  =  1
###################
mean:    327.81806696146555
median:  110.0
mode :   100.0
StDev:   1288.8296456971598

Categeory  =  2
###################
mean:    70.10467706013362
median:  50.0
mode :   47.0
StDev:   69.47337105763614

Categeory  =  3
###################
mean:    56.873333333333335
median:  47.0
mode :   50.0
StDev:   52.47280508161027

Categeory  =  4
###################
mean:    58.3725
median:  38.5
mode :   70.0
StDev:   74.12604702795065



###### Ratings ########
Categeory  =  1
###################
mean:    86.58399511301161
median:  87.0
mode :   86
StDev:   4.039210375758337

Categeory  =  2
###################
mean:    87.89755011135857
median:  89.0
mode :   90
StDev:   5.170591485127886

Categeory  =  3
###################
mean:    83.23
median:  84.0
mode :   84
StDev:   4.0873121242863

Categeory  =  4
###################
mean:    85.465
median:  86.0
mode :   88
StDev:   5.462413827615096



###### % Alcohol ########
Categeory

It appears that price and percent alcohol don't vary much. That makes sense as most whisk(e)y's generally sit between 40% and 61%. Canadian whiskey tends to be 40% whereas the upper limit for most scotches is 52. However, many overproof american whiskeys like Bottled in Bond and Cask Strengths may also sit around the scotch level. Only the really insane stuff like Booker's reaches the vaunted 61%. 

> Given the lack of variety and existence of null values in the validation set, I will remove the percent_alcohol and rating feature

Price is staying despite it's null values because it shows a marked difference between classes, specifically identifying Scotch. 

This can then be imputed based on the next engineered features. 

In [289]:
df = df.drop(columns = ['ratingValue','pert_alcohol'])

## Identifying Key Terms

Given these are reviews, there is a pretty significant amount of leakage in the desciption text. They often provide keywords that are associated with the specified field. 

#### Scotch

The first row that is classified as a Scotch is this one. It lists the name Bowmore which is a distiller from the isle of Islay that utilizes sherry finished on their characteristically smokey scotches. 

In [307]:
df[df['category'] == 1]['description'][1]

"There have been some legendary Bowmores from the mid-60s and this is every bit their equal. All of them share a remarkable aroma of tropical fruit, which here moves into hallucinatory intensity: guava, mango, peach, pineapple, grapefruit. There’s a very light touch of peat smoke, more a memory of Islay than the reality. Concentrated; even at low strength the palate is silky, heady, and haunting, and lasts forever in the dry glass. A legend is born. (Eight bottles only for the U.S.) Editor's Choice."

#### Bourbon

The Bourbon is harder to identify. It relied on name recongnition. Either the distiller or the distillery will be mentioned. 

In [309]:
df[df['category'] == 2]['description'][2]

"This bottling celebrates master distiller Parker Beam's 50 years of service by including whiskey from each of the past five decades. This is a fabulous whiskey: seamless and incredibly complex, with an impeccable marriage of youth and maturity. It’s also very even-keeled throughout -- quite different than last year’s equally impressive PHC, a 27 year old, whose personality was more like an exhilarating old wooden rollercoaster ride (and also brandished more oak).\xa0Look for candied citrus, nectarine, blueberry, and sultana anchored by a nougat center, laced with honeyed vanilla and orange creamsicle. There’s a dusting of cocoa powder, brittle mint, and cinnamon, too. Tobacco leaves, polished leather, and teasing bourbon barrel char round out the palate, emerging more prominently towards a warming finish. A classic!"

#### Craft Whiskey

Craft distilleries are harder to identify. Often times they will source from all over the place. This will lead to a hodgepodge of distilleries. If anything this is the wildcard. It may be best to ignore this feature alltogether in case any feature engineering confuses the model. 

In [311]:
df[df['category'] == 3]['description'][391]

'A sourced whiskey of 95% corn, finished in wine barrels from winemaker Dave Phinney. This makes a super first impression, with Bit-O-Honey candy, eucalyptus, black cherry, cinnamon hearts, violet candies, and sandalwood. The flavors pour layered and complex, with clove-studded orange, flickers of rye spice, and pure, crystalline sweetness balanced with lemony lift. Laser-like spice, sweet caramel corn, and more floral notes dance across the long finish.'

#### Canadian Whiskey

Canada sources a lot of whiskey to the United States craft distilleries. I have to be careful about classifying Rye or Canada as it is often mentioned in the craft classes. On top of that, almost all Canadian whiskeys are blends of multiple distilleries. it might be good to identify Canadian Brands rather than regions or distilleries. 

In [319]:
df[df['category'] == 4]['description'][93]

'Corby’s latest Lot 40, this one undated, comes from the same distillation batch as the 2012 release, but with a couple of extra years in wood. The familiar flavors are all there: dustiness, sour rye, hard wet slate, floral notes, exotic fruits, sweet spices, and biting white pepper. Over these, time has sprinkled licorice root, dried dates, oatmeal porridge, vanilla, hints of bike tires, and mango peels. Flavors remain fully integrated with faint tannins underscoring a long sour-rye finish. Value Pick.'

### Keyword Identification

There are a lot of Scotch keywords so I am going to scrape those. The Canadian and Bourbon keywords I will recite by memory. 

In [385]:
#res = requests.get(URL).text
URL = requests.get('https://en.wikipedia.org/wiki/List_of_whisky_distilleries_in_Scotland').text

soup = BeautifulSoup(URL,'html.parser')

scotch_references = []

soup = BeautifulSoup(URL,'html.parser')
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        distillery = data[0].a.text
    
    except AttributeError:pass
    except IndexError:pass
    scotch_references.append(distillery)
    
other_scotch_references = ['loch','speyside','campbeltown',
                           'highland','islay','orkney','skye']

for keyword in other_scotch_references:
    scotch_references.append(keyword)
    
bourbon_references = ['Kentucky','Bourbons','Lexington','Sazerac','Bardstown',
                      'Buffalo Trace','1792','Colonel','Barton','Dixie','Stagg',
                      'Winkle','Bowman','Blanton','Eagle Rare','E.H. Taylor',
                      'Elmer T. Lee','Weller','Heaven Hill','Brown Forman', 'Beam',
                      'Elijah Craig','Evan Williams','Old Fitzgerald','Basil Hayden']

canadian_references = ['Alberta', 'Canadian Mist', 'Crown Royal','Forty Creek','Highwood'
                       'Pendleton','Pemberton','Hiram Walker','Lot 40','Wiser','Seagram']

bourbon_references = [x.lower() for x in bourbon_references]
canadian_references = [x.lower() for x in canadian_references]
scotch_references = [x.lower() for x in scotch_references]

references = [bourbon_references, 
              canadian_references, 
              scotch_references]

In [386]:
scotch = []
bourbon = []
canadian = []

for i in np.arange(len(df)):
    for k in references:
        ref_count = 0
        for j in k:
            if j in df['description'][i].lower():
                ref_count += 1
        if k == bourbon_references:
            bourbon.append(ref_count)
        elif k == scotch_references:
            scotch.append(ref_count)
        else:
            canadian.append(ref_count)

df['scotch_reference'] = scotch
df['bourbon_reference'] = bourbon
df['canadian_reference'] = canadian

#### Baseline Models

Now that references to the data have been engineered into features, it is time to run some cross-validation and baseline models. 

In [90]:
def cleaner():
    
    cleaned_entry = []
    
    for i in np.arange(len(df)):
        
        sample = df['description'][i]
        soup = BeautifulSoup(sample, "html.parser")
        
        sample = (re.sub(r'[^a-zA-Z ^0-9]', '', 
                         soup.text))
            
        cleaned_entry.append(sample)

    return cleaned_entry



In [425]:
b_line = df[['description','category']].dropna()
X = b_line['description']
y = b_line['category']

In [411]:
vect = TfidfVectorizer(stop_words='english', 
                        max_features = 10000)

sgdc  = SGDClassifier()

xgbc = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                        colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
                        max_depth=17, min_child_weight=1, missing=None, n_estimators=100,
                        n_jobs=-1, nthread=None, objective='multi:softmax',num_class=4,
                        random_state=5, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                        seed=42, silent=True, subsample=1, eval_metric='merror')

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

pipe.fit(X, y)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=10000,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
           

In [412]:
score_1 = (cross_val_score(pipe, X, y, 
                          cv = 10, 
                          scoring = 'accuracy',
                          n_jobs = -1,
                          verbose = 10)).mean()

print('cross validation baseline: ', score_1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.4s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.5s remaining:    0.2s


cross validation baseline:  0.9392697679775773


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.7s finished


In [435]:
b_2 = df[['author', 'price', 'scotch_reference', 
          'bourbon_reference', 'canadian_reference', 
          'category']].dropna()

X = b_2.drop(columns = 'category')
y = b_2['category']


encoder = ce.OrdinalEncoder()

pipe_2 = Pipeline([('ord', encoder), ('clf', xgbc)])

pipe_2.fit(X, y)

Pipeline(memory=None,
         steps=[('ord',
                 OrdinalEncoder(cols=['author'], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[{'col': 'author',
                                          'data_type': dtype('O'),
                                          'mapping': John Hansell               1
Dave Broom                 2
Fred Minnick               3
Davin de Kergommeaux       4
Gavin Smith                5
Dominic Roskrow            6
Geoffrey Kleinman          7
Jonny McCormick            8
Susannah Skiver Barton     9
Lew Bryson                10
Jeffery Lindenmut...
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='merror',
                               gamma=0, learning_rate=0.1, max_delta_step=0,
                               max_depth=17, min_child_weight=1, missing=None,
            

In [436]:
score_2 = (cross_val_score(pipe_2, X, y, 
                          cv = 10, 
                          scoring = 'accuracy',
                          n_jobs = -1,
                          verbose = 10)).mean()

print('second validation baseline: ', score_2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    1.5s remaining:    3.5s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    1.5s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    1.5s remaining:    0.7s


second validation baseline:  0.9245274059122057


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.4s finished


It appears that the vectorized description model performed slightly better by utilizing a stochastic gradient descent classifier. The other features when seem to peform almost as good as the nlp vectorization when run through a gradient boosting tree ensemble. Now I am going to try to engineer a new dataframe that combines both. I can either use a feature union or concatentate the dataframe without a pipeline. 

In [438]:
tfidf = TfidfVectorizer(stop_words='english', 
                        max_features = 5000, 
                        min_df=5)

dtm = tfidf.fit_transform(df['description'])

2874

## Lecture Notes

In [42]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100,
                   algorithm='randomized',
                   n_iter=10) 

In [43]:
# LSI

lsi = Pipeline([('vect', vect), ('svd', svd)])

In [64]:
pipe = Pipeline([('lsi',lsi),('clf',xgbc)])

In [65]:
pipe.fit(X,y)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=10000, min_df=1,
                                                  ngram_range=(1, 1), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_

In [108]:
y_pred = pipe.predict(test['description'])

In [109]:
sample_submission['category'] = y_pred.astype(int)
sample_submission.to_csv('fifth.csv', index=False)
!kaggle competitions submit -c ds3-which-whisky -f fifth.csv -m "fifth attempt!"

100%|██████████████████████████████████████| 1.91k/1.91k [00:00<00:00, 2.10kB/s]
Successfully submitted to DS3 Which Whisky

In [97]:
ban_vector = doc.vector

In [99]:
print(len(ban_vector))

300


In [100]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [101]:
sgdc.fit(get_word_vectors(X), y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [102]:
y_pred = sgdc.predict(get_word_vectors(test['description']))

In [104]:
len(y_pred)

288

In [110]:
df['description'][0]

'A marriage of 13 and 18 year old bourbons A mature yet very elegant whiskey with a silky texture and so easy to embrace with a splash of water Balanced notes of honeyed vanilla soft caramel a basket of complex orchard fruit blackberry papaya and a dusting of cocoa and nutmeg smooth finish Sophisticated stylish with welldefined flavors A classic'

In [111]:
####TODO####

"""
- CLEANUP CODE

- FILL NA'S WITH KEYWORDS

- NESTED FEATURE UNIONS

- CROSS VALIDATION/OPTIMIZATION

-PROFIT

"""

"\n- CLEANUP CODE\n\n- FILL NA'S WITH KEYWORDS\n\n- NESTED FEATURE UNIONS\n\n- CROSS VALIDATION/OPTIMIZATION\n\n-PROFIT\n\n"

In [121]:
df[df['category'].isnull() == True]['description']

Int64Index([   4,    9,   25,   43,   52,   70,  102,  106,  115,  152,
            ...
            2766, 2780, 2789, 2790, 2809, 2822, 2828, 2830, 2839, 2853],
           dtype='int64', length=288)

In [125]:
df[df['category'].isnull() == True]['description'][25]

'Classic sherry cask-aged Macallan. Antique amber with hints of ruby. Thickly textured, complex, and quite deep, with notes of toffee, ripe pit fruit, raisin, apple pie, dried spice (cinnamon, clove, ginger, and mint), and a wisp of smoke. An exceptional, multi-faceted Macallan!'

In [ ]:
canada_indicator = ['Cana']
scotch 